# Evaluate UMLS

In [3]:
import pandas as pd

In [7]:
umls_predicitons = pd.read_csv(f'../outputs/eval_demographics-fewshot_gpt-4o-2024-05-13_minc-40_maxc-4000_umls.csv')
annotations = pd.read_csv(f'../annotations/combined_pd.csv')

In [8]:
patients = annotations[annotations.group_name == 'patients']

In [9]:
# For each PMCID separatey, get the best match for each UMLS code, without replacement (i.e. only one match per UMLS code)

for pmcid, group in patients.groupby('pmcid'):
    group_preds = umls_predicitons[umls_predicitons.pmcid == pmcid]

    if len(group_preds) == 0:
        continue

    used_group_ixs = []
    for row in group.itertuples():
        if pd.isnull(row.umls):
            continue

        correct = [c.strip() for c in row.umls.split(', ')]
        
        matches = group_preds[(group_preds['umls_cui'].isin(correct)) & (group_preds['group_ix'].isin(group) == False)]
        if len(matches) > 0:
            m =  matches[matches['umls_prob'] == matches['umls_prob'].max()].iloc[0]
            used_group_ixs.append(m['group_ix'])
            
            patients.at[row.Index, 'umls_match'] = 1
            patients.at[row.Index, 'umls_prob'] = m['umls_prob']
            patients.at[row.Index, 'diagnosis_pred'] = m['diagnosis']
        else:
            patients.at[row.Index, 'umls_match'] = 0# For each PMCID separatey, get the best match for each UMLS code, without replacement (i.e. only one match per UMLS code)



/tmp/ipykernel_1458306/1771556649.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients.at[row.Index, 'umls_match'] = 0# For each PMCID separatey, get the best match for each UMLS code, without replacement (i.e. only one match per UMLS code)
/tmp/ipykernel_1458306/1771556649.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients.at[row.Index, 'umls_prob'] = m['umls_prob']
/tmp/ipykernel_1458306/1771556649.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [10]:
patients['umls_match'].mean()

0.7652173913043478

In [17]:
qualitative_correct_n = 15
(patients['umls_match'].sum() + qualitative_correct_n)/ (patients.umls_match.isna() == False).sum()

0.8956521739130435

#### Look at UMLS with no prediction at all

In [18]:
gpt_4_o = pd.read_csv('../outputs/eval_demographics-fewshot_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')
isna = patients[patients.umls_match.isna()].pmcid.unique()
gpt_4_diagna = gpt_4_o[gpt_4_o.pmcid.isin(isna)]


In [23]:
# Fully missing from GPT_4 predictions
set(isna) - set(gpt_4_diagna.pmcid.unique())

{4732188, 4936600, 6290711, 7275020, 8459240, 8785614, 8933759, 10870473}

In [20]:
for pmcid in isna:
    if pmcid not in gpt_4_diagna.pmcid.unique():
        continue
    print(pmcid)
    display(gpt_4_diagna[gpt_4_diagna.pmcid == pmcid])
    display(patients[patients.pmcid == pmcid])
    print()

4522562


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_range,age_minimum,age_maximum,age_median,imaging_sample,pmcid,rank,start_char,end_char
362,8.0,NaN,healthy,NaN,4.0,4.0,31.75,NaN,NaN,NaN,NaN,yes,4522562.0,0,34627,36506


,Unnamed: 0,group_name,subgroup_name,project_name,annotator_name,pmcid,umls,diagnosis,count,male count,age mean,female count,age minimum,age maximum,age median,umls_match,umls_prob,diagnosis_pred
0,120,patients,rpc,participant_demographics,Jerome_Dockes,4522562,C4049190,thermo-coagulation lesions in the right parah...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,116,patients,chronic lumbar back-pain,participant_demographics,Jerome_Dockes,4522562,C0457949,chronic lumbar back-pain,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,117,patients,epilepsy,participant_demographics,Jerome_Dockes,4522562,C0014544,epilepsy,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,118,patients,refractory epilepsy,participant_demographics,Jerome_Dockes,4522562,C1096063,refractory epilepsy,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,121,patients,lh,participant_demographics,Jerome_Dockes,4522562,C4049190,thermo-coagulation lesions in the left hippoca...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,122,patients,lph,participant_demographics,Jerome_Dockes,4522562,C4049190,thermo-coagulation lesions in the left parahip...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,119,patients,rh,participant_demographics,Jerome_Dockes,4522562,C4049190,thermo-coagulation lesions in the right hippoc...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



5394595


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_range,age_minimum,age_maximum,age_median,imaging_sample,pmcid,rank,start_char,end_char
171,60.0,NaN,healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,5394595.0,0,5950,6820
172,17.0,bronchial asthma,patients,GCBT,7.0,10.0,50.76,NaN,NaN,NaN,NaN,no,5394595.0,0,5950,6820


...

,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_range,age_minimum,age_maximum,age_median,imaging_sample,pmcid,rank,start_char,end_char
385,21.0,"PTSD, depression, anxiety, conduct disorder an...",patients,childhood abuse,15.0,6.0,17.5,NaN,NaN,NaN,NaN,yes,5104469.0,0,12106,15360
386,20.0,NaN,healthy,psychiatric controls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,5104469.0,0,12106,15360
387,27.0,NaN,healthy,healthy controls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,5104469.0,0,12106,15360


,Unnamed: 0,group_name,subgroup_name,project_name,annotator_name,pmcid,umls,diagnosis,count,male count,age mean,female count,age minimum,age maximum,age median,umls_match,umls_prob,diagnosis_pred
125,126,patients,severe childhood abuse,participant_demographics,Jerome_Dockes,5104469,C5828513,Severe Childhood Abuse,21,15.0,17.5,6.0,NaN,NaN,NaN,0.0,NaN,NaN
263,127,patients,psychiatric controls,participant_demographics,Jerome_Dockes,5104469,NaN,NaN,19,9.0,16.9,10.0,NaN,NaN,NaN,NaN,NaN,NaN



11033924


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_range,age_minimum,age_maximum,age_median,imaging_sample,pmcid,rank,start_char,end_char
12,122.0,relapsing-remitting MS,patients,NaN,38.0,84.0,37.69,18-65,18.0,65.0,NaN,yes,11033924.0,0,7923,9510
13,97.0,NaN,healthy,NaN,41.0,56.0,38.10,18-65,18.0,65.0,NaN,yes,11033924.0,0,7923,9510


,Unnamed: 0,group_name,subgroup_name,project_name,annotator_name,pmcid,umls,diagnosis,count,male count,age mean,female count,age minimum,age maximum,age median,umls_match,umls_prob,diagnosis_pred
442,464,patients,_,participant_demographics,ju-chi_yu,11033924,NaN,NaN,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
